# Facade Generator

Setup steps: import builtin and 3rd-party libraries, load environment variables (needed for database connection), and import modules from `ud-tools-core`.

In [39]:
import os
import sys

from dotenv import load_dotenv
from shapely.geometry import shape
from shapely.ops import transform
import shapely.wkt

In [40]:
load_dotenv("../.env.local")

True

In [41]:
home_path = os.path.expanduser('~')
module_path = os.path.join(home_path, 'git/ud-tools-core/python/')
if module_path not in sys.path:
    sys.path.append(module_path)

In [42]:
from models.zoninglot import ZoningLot
from models.site import Site
from models.scenario import Scenario

In [37]:
# get query bounds and area centroid from default
# aoi = shapely.wkt.loads('POLYGON ((1009405.90154069 234515.39897036, 1009930.58999844 234367.63467916, 1009849.1944143 234052.07026066, 1009314.48803851 234194.82559284, 1009405.90154069 234515.39897036))')

# define common query params for study area
# params = {
#     'center_x': aoi.representative_point().x,
#     'center_y': aoi.representative_point().y,
#     'bounds': aoi.wkt
# }

Set up a `ZoningLot` object, we need this to get the buildings that sit on it and then sort their facades based on orientation relative to the lot.

In [44]:
my_scenario = Scenario('A')
my_site = Site('2', ['2025540049'])
zl = ZoningLot(my_scenario, my_site)
# zl.define_zone('R6A')

conn strhost=0.0.0.0 port=25432 user=user dbname=nyc password=1234
conn strhost=0.0.0.0 port=25432 user=user dbname=nyc password=1234
conn strhost=0.0.0.0 port=25432 user=user dbname=nyc password=1234


In [46]:
zl.baseplane_elevation # approximate baseplane ht in feet
zl.edges # dictionary of lot edges, classified by type
# zl.type # not yet implemented

{'ids': [1320, 1322, 2514, 2518, 3723],
 'front': [{'id': 1322,
   'geom': 'LINESTRING(62.6437825600151 40.73067328101024,28.617264691973105 -43.95516541833058)',
   'azimuth': 3.523647227744336,
   'length': 91.266068143834,
   'front': True,
   'streetname': 'Cypress Avenue',
   'streetwidth': Decimal('80'),
   'baseplane': 46}],
 'rear': [{'id': 2518,
   'geom': 'LINESTRING(-71.07348113856278 -1.569100888911635,-36.019409235334024 81.01962753897533)',
   'azimuth': 0.4013973760000926,
   'length': 89.72004246728818,
   'front': False,
   'streetname': None,
   'streetwidth': None},
  {'id': 3723,
   'geom': 'LINESTRING(-36.019409235334024 81.01962753897533,-35.42330461344682 82.42678400641307)',
   'azimuth': 0.40070420672465623,
   'length': 1.5282113872390695,
   'front': False,
   'streetname': None,
   'streetwidth': None}],
 'side': [{'id': 2514,
   'geom': 'LINESTRING(28.617264691973105 -43.95516541833058,-71.07348113856278 -1.569100888911635)',
   'azimuth': 5.114408284021454

In [47]:
bldgs = zl.load_buildings(use_as_envelope=False) # load_buildings returns a list, since possibly multiple buildings on the zoning lot

conn strhost=0.0.0.0 port=25432 user=user dbname=nyc password=1234


In [48]:
# get faces from building, process and render
bldg = bldgs[0]
bldg.bin
bldg.envelope
bldg.faces

walls = [f for f in bldg.faces if f['wall']]
lot_line_walls = [w for w in walls if w['lot_edge'] is not None]

In [49]:
from geometry.conversion import wkt_to_occ, curve_to_face
from geometry.measure import surface_area

for i, w in enumerate(walls):
    wall_crv = wkt_to_occ(w['geom'])
    wall_face = curve_to_face(wall_crv)
    area = surface_area(wall_face)
    print(f'{i} --- {area}')

True
0 --- 138.45346878788732
True
1 --- 205.39905642457072
True
2 --- 138.4534687878874
True
3 --- 205.39905642457072
True
4 --- 188.927828131435
True
5 --- 94.6491326531043
True
6 --- 153.21049384010604
True
7 --- 111.40191054020106
True
8 --- 88.97864007397796
True
9 --- 92.58121284103768
True
10 --- 301.07990925801914
True
11 --- 787.159289806189
True
12 --- 596.6143637761147
True
13 --- 2507.2789772970077
True
14 --- 2026.7023394957016
True
15 --- 1580.3035319476783
True
16 --- 724.7490152292185
True
17 --- 1580.2950615403934
True
18 --- 2123.474740746866
True
19 --- 2276.3817379443476
True
20 --- 666.2259864242457
True
21 --- 510.5652601588203
True
22 --- 137.74738634446607
True
23 --- 207.18212397445365
True
24 --- 343.2065556709687
True
25 --- 1689.0750454662784
True
26 --- 50.43308146247546
True
27 --- 50.43308146247546
True
28 --- 2488.3471280907866
True
29 --- 8.896378911064463
True
30 --- 1376.6893524463512
True
31 --- 1226.701766952054
True
32 --- 1038.1274956332168
True
3

In [50]:
# select a wall manually
wall = walls[28]

using this wall, set up collection of (2d?) geometry objects needed for generation of the facade. first tried using oriented bounding box to establish, but turns out it's easier to create a face from the boundary curve and use an adaptor to access the generated plane and parameterization of the face rather than setting up from scratch (bbox params)

In [51]:
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface

In [52]:
from maths.domain import value_in_domain, scale_list, consecutive_domains, reparameterize_list, domain_length
from geometry.conversion import curve_to_wire

In [53]:
from OCC.Core.GeomAPI import (
    GeomAPI_ProjectPointOnCurve,
    geomapi_To2d,
    geomapi_To3d,
)

In [54]:
wall_crv = wkt_to_occ(wall['geom'])

True


In [55]:
# to generate these curves, we'll use UV parameters on a surface
# the surface is provided by an adaptor from a face generated from the curve
wall_face = curve_to_face(wall_crv)
wall_srf = BRepAdaptor_Surface(wall_face)
wall_pln = wall_srf.Plane()

In [17]:
# also set up a 2d outline curve to use for the drawing
wallcrv_2d = geomapi_To2d(wall_crv, wall_pln)

# this is the collection of 2d curves we'll build up as the facade drawing
crvs_2d = [wallcrv_2d]

In [57]:
# information about the face in 3d space
u_min = wall_srf.FirstUParameter()
u_max = wall_srf.LastUParameter()
v_min = wall_srf.FirstVParameter()
v_max = wall_srf.LastVParameter()
width = abs(u_min) + abs(u_max)
height = abs(v_min) + abs(v_max)
elevation = wall_srf.Value(0, 0).Z()
bp = zl.baseplane_elevation
ground = bp - elevation

print(f'{width} x {height}')
print(f'center at {elevation}, baseplane at {bp} ft')
print(f'ground at {ground} in local space')

36.88878528832899 x 60.758499999996275
center at 76.3104500000045, baseplane at 46 ft
ground at -30.310450000004494 in local space


In [58]:
# generate u spans proportionally across domain
pattern = [1, 2, 2, 2, 1]
u_spans = consecutive_domains((-18.0, 18.0), pattern, False, False)
u_spans

[(-18.0, -13.5), (-13.5, -4.5), (-4.5, 4.5), (4.5, 13.5), (13.5, 18.0)]

In [59]:
# generate v spans in absolute units (floor-floor heights)
pattern = [15, 10, 10, 10]
v_spans = consecutive_domains((-30, 30), pattern, False, True)
v_spans

[(-30, -15), (-15, -5), (-5, 5), (5, 15)]

In [60]:
uv_spans = (u_spans, v_spans)

In [61]:
def coords_from_grid(local_coords, grid, cell, absolute=False):

    local_u = grid[0][cell[0]]
    local_v = grid[1][cell[1]]

    coords = coords_from_grid_cell(local_coords, local_u, local_v)

    return coords


def coords_from_grid_cell(local_coords, u_domain, v_domain, absolute=False):
    grid_u = value_in_domain(local_coords[0], u_domain, param_domain=(-1, 1))
    grid_v = value_in_domain(local_coords[1], v_domain, param_domain=(0, 1))
    
    coords = (grid_u, grid_v)

    return coords


In [62]:
# get the midpoint of each cell in the grid
# checking to make sure the element fits
coords = []

for u in uv_spans[0]:
    u_length = domain_length(u)
    for v in uv_spans[1]:
        v_length = domain_length(v)
        if u_length > 5.0 and v_length > 5.0:
            coord = coords_from_grid_cell((0, 0.8), u, v)
            coords.append(coord)

coords

[(-9.0, -18.0),
 (-9.0, -7.0),
 (-9.0, 3.0),
 (-9.0, 13.0),
 (0.0, -18.0),
 (0.0, -7.0),
 (0.0, 3.0),
 (0.0, 13.0),
 (9.0, -18.0),
 (9.0, -7.0),
 (9.0, 3.0),
 (9.0, 13.0)]

In [63]:
from geometry.conversion import coords_to_curve
from OCC.Core.gp import gp_Vec2d, gp_Trsf2d


In [64]:
# for each point in the generated coordinates from the grid,
# add a generated shape to the "drawing" (list of 2d curves)
for c in coords:
    window = coords_to_curve([(-2, 0), (2, 0), (2, -4), (-2, -4), (-2, 0)], z=False)
    vec = gp_Vec2d(*c) # translate from 0, 0 to the grid point from above
    window.Translate(vec)
    crvs_2d.append(window)

In [65]:
# flip back to 3d space using face plane
crvs_3d = [geomapi_To3d(c, wall_pln) for c in crvs_2d]

In [66]:
# convert 3d curves to wires for rendering
crv_wires = [curve_to_wire(c) for c in crvs_3d]

In [28]:
# OCC renderer
# renderer = JupyterRenderer()

In [30]:
# display the current envelope, based on the existing building(s)
# renderer.DisplayShape(crv_wires[5], render_edges=True, topo_level="Face", shape_color="#abdda4", update=True)

## Export to STEP file

(can be opened with Rhino)

In [31]:
from connectors.step import shapes_to_step

shapes_to_step(crv_wires, "/Users/carsten/Desktop/facades.stp")

1